# Outlier The Year in Fabrics - 2020

Accompaniment to Abe's [The Year in Fabrics - 2020](https://www.reddit.com/r/Outlier/comments/kur28i/the_year_in_fabrics_2020/)

In [1]:
import pandas as pd
import numpy as np
import scipy
import os
import os.path
from IPython.display import display
import utils
import plots
import altair as alt

df_all = utils.outlier_df('../../outlier-data.csv')

In [2]:
# Filter to Clothes
def type_to_kind(drop_type):
    if drop_type == "T-Shirt" or drop_type == "Shirt" or drop_type == "Tank Top":
        return "Top"
    if drop_type == "Pants" or drop_type == "Shorts":
        return "Bottom"
    return "Layer"


def row_to_subkind(row):
    drop_type = row['Type']
    if drop_type == "T-Shirt" or drop_type == "Shirt":
        pass
    elif drop_type == "Pants" or drop_type == "Shorts":
        pass
    elif drop_type == "Tank Top":
        drop_type = 'T-Shirt'
    else:
        drop_type = "Layer"
    exp = "Exp" if row['Exp'] else "Regular"
    return "{}/{}".format(drop_type, exp)


def object_type_to_kind(drop_type):
    if drop_type == "Backpack" or drop_type == "Bag" or drop_type == "Dopp Kit" \
       or drop_type == "Dufflebag" or drop_type == "Pouch":
        return "Bag"
    if drop_type == "Balaclava" or drop_type == "Bandana" or drop_type == "Scarf":
        return "Scarf/Bandana"
    if drop_type == "Cap" or drop_type == "Hat" or drop_type == "Hood":
        return "Headgear"
    if drop_type == "Bathrobe" or drop_type == "Robe" or drop_type == "Towel" \
       or drop_type == "Outerwear" or drop_type == "Keychain" or drop_type == "Socks":
        return "Misc"
    if drop_type == "Mask" or drop_type == "Belt":
        return drop_type     
    return "Misc"

# Define some reused frames
all_products_df = utils.drops_to_products(df_all, ['Year', 'Product'])
df_all['ColorCount'] = df_all['Colors'].str.split(',').dropna().apply(lambda d: len(d))

df_clothes = pd.DataFrame(df_all.loc[df_all['Category'] == 'Clothes', :])
df_clothes['Type'] = df_clothes['Type'].replace({'Sweatshirt':'Sweater'})
df_clothes["Kind"] = df_clothes['Type'].apply(type_to_kind)
df_clothes["Subkind"] = df_clothes.apply(row_to_subkind, axis=1)
clothes_products_df = utils.drops_to_products(df_clothes, ['Year', 'Product'])

df_objects = pd.DataFrame(df_all.loc[df_all['Category'] == 'Objects', :])
df_objects["Kind"] = df_objects['Type'].apply(object_type_to_kind)
objects_products_df = utils.drops_to_products(df_objects, ['Year', 'Product'])


kind_sort_order = ['Bottom', 'Top', 'Layer']
object_kind_sort_order = ['Bag', 'Mask', 'Scarf/Bandana', 'Headgear', 'Belt', 'Other']
year_sort = list(range(2020, 2012, -1))

In [3]:
df_2020 = df_all[df_all['Release'].dt.year == 2020]

In [4]:
from plots import product_summary_html
from IPython.display import HTML

def fabric_summary_display(df_all, fabric):
    # Order by number of drops, not alphabetically
    # products = sorted(products)
    products = df_all[df_all['Fabric'] == fabric].groupby('Product').count()['Type'].sort_values(ascending=False).index
    if len(products) < 1:
        return
    htmls = [product_summary_html(df_all, p) for p in products]
    fragments = [f"<div class='product'>{h.data}</div>" for h in htmls]
    row = f"<h3>{fabric}</h3><div class='fabric-products' style='display: flex'>{' '.join(fragments)}</div>"
    display(HTML(row))

In [5]:
fabrics = sorted(df_2020['Fabric'].unique())
for f in fabrics:
    fabric_summary_display(df_2020, f)

,Value
last drop,2020
price,198
number of drops,2
fabric,Albini Merino Broadcloth
number of colors,5
colors,"Blue, Bluegray, Gray, Plumsmoke, Rich Navy"
,Value
last drop,2020
price,275
number of drops,1


,Value
last drop,2020
price,248
number of drops,1
fabric,Bombtwill
number of colors,1
colors,Black
,Value
last drop,2020
price,225
number of drops,1


,Value
last drop,2020
price,110
number of drops,1
fabric,Brut Cotton
number of colors,2
colors,"Navy Space, White"
,Value
last drop,2020
price,110
number of drops,1


,Value
last drop,2020
price,125
number of drops,5
fabric,Cottonweight Merino
number of colors,17
colors,"GD Black, GD Charcoal, GD Dry Rose, GD Dry Tan, GD Fuchsia, GD Gray Light, GD Green, GD High Blue, GD Mauve Light, GD Mineral Pink, GD Plumsmoke, GD Racing Green, GD Racinng Green, GD Sage, GD Seeing Red, GD Yellow, Washed White"
,Value
last drop,2020
price,160
number of drops,2


,Value
last drop,2020
price,75
number of drops,1
fabric,Doublefine Merino
number of colors,1
colors,Black


...

,Value
last drop,2020
price,48
number of drops,6
fabric,Ultrasuede
number of colors,10
colors,"Black, Camel, Copper, Dark Green, Espresso, Fuchsia, Light Gray, Maroon, Purple, Tan"


,Value
last drop,2020
price,375
number of drops,1
fabric,Warmform Merino
number of colors,1
colors,Black


,Value
last drop,2020
price,198
number of drops,2
fabric,Workcloth
number of colors,5
colors,"Black, Blacktint Green, Bluetint Gray, Charcoal, Sandshadow"
,Value
last drop,2020
price,198
number of drops,1


,Value
last drop,2020
price,250
number of drops,1
fabric,Workcloth 320
number of colors,1
colors,Charcoal


,Value
last drop,2020
price,275
number of drops,1
fabric,“60/30”
number of colors,4
colors,"Black Navy, Desert Tan, Olive, Steel Gray"
